In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from pygrinder import mcar
from pypots.data import load_specific_dataset

I:\work\SANNI\venv\lib\site-packages\pypots\nn\modules\reformer\local_attention.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
I:\work\SANNI\venv\lib\site-packages\pypots\nn\modules\reformer\local_attention.py:102: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


In [3]:
data = load_specific_dataset('physionet_2012')  # PyPOTS will automatically download and extract it.


2024-09-10 21:26:26 [INFO]: Loading the dataset physionet_2012 with TSDB (https://github.com/WenjieDu/Time_Series_Data_Beans)...
2024-09-10 21:26:26 [INFO]: Starting preprocessing physionet_2012...
2024-09-10 21:26:26 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-09-10 21:26:26 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-09-10 21:26:26 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2024-09-10 21:26:26 [INFO]: Loaded successfully!
2024-09-10 21:27:04 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2024-09-10 21:27:05 [INFO]: 69309 values masked out in the val set as ground truth, take 10.04% of the original observed values
2024-09

In [4]:
data.keys()

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'val_X_ori', 'test_X_ori'])

In [5]:
X = data['train_X']
num_samples = len(X)
# X = X.drop(['RecordID', 'Time'], axis = 1)
X = X.reshape(-1,37)
X = StandardScaler().fit_transform(X)
X = X.reshape(num_samples, 48, -1)
X_ori = X  # keep X_ori for validation
X = mcar(X, 0.1)

In [6]:
print(X.shape)  # (11988, 48, 37), 11988 samples and each sample has 48 time steps, 37 features

(7671, 48, 37)


In [7]:
from pypots.imputation import BRITS
from BRITS.PypotsBrits import BRITS as myBrits

In [20]:
saits = BRITS(n_steps=48, n_features=37, epochs=10,rnn_hidden_size=10)


2024-09-10 21:39:24 [INFO]: No given device, using default device: cuda
2024-09-10 21:39:24 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2024-09-10 21:39:24 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 19,628
2024-09-10 21:39:24 [INFO]: No given device, using default device: cuda
2024-09-10 21:39:24 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2024-09-10 21:39:24 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 19,628


In [21]:
dataset = {"X": X[:100]}  # X for model input
saits.fit(dataset)  # train the model on the dataset


2024-09-10 21:39:37 [INFO]: Epoch 001 - training loss: 1.3701
2024-09-10 21:39:44 [INFO]: Epoch 002 - training loss: 1.4052
2024-09-10 21:39:48 [INFO]: Epoch 003 - training loss: 1.4267
2024-09-10 21:39:51 [INFO]: Epoch 004 - training loss: 1.3724
2024-09-10 21:39:55 [INFO]: Epoch 005 - training loss: 1.3211
2024-09-10 21:39:58 [INFO]: Epoch 006 - training loss: 1.3225
2024-09-10 21:40:01 [INFO]: Epoch 007 - training loss: 1.3225
2024-09-10 21:40:04 [INFO]: Epoch 008 - training loss: 1.3668
2024-09-10 21:40:07 [INFO]: Epoch 009 - training loss: 1.3366
2024-09-10 21:40:10 [INFO]: Epoch 010 - training loss: 1.2967
2024-09-10 21:40:10 [INFO]: Finished training. The best model is from epoch#10.


In [48]:
dataset = {"X": X[:100]}  # X for model input
myBritrs = myBrits(n_steps=48, n_features=37, epochs=10,rnn_hidden_size=10)
myBritrs.fit(dataset) 

2024-09-10 21:47:30 [INFO]: No given device, using default device: cuda
2024-09-10 21:47:30 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2024-09-10 21:47:30 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 19,628
2024-09-10 21:47:34 [INFO]: Epoch 001 - training loss: 1.3995
2024-09-10 21:47:37 [INFO]: Epoch 002 - training loss: 1.3933
2024-09-10 21:47:40 [INFO]: Epoch 003 - training loss: 1.3421
2024-09-10 21:47:43 [INFO]: Epoch 004 - training loss: 1.3408
2024-09-10 21:47:48 [INFO]: Epoch 005 - training loss: 1.5188
2024-09-10 21:47:53 [INFO]: Epoch 006 - training loss: 1.3456
2024-09-10 21:47:55 [INFO]: Epoch 007 - training loss: 1.3334
2024-09-10 21:47:58 [INFO]: Epoch 008 - training loss: 1.3464
2024-09-10 21:48:01 [INFO]: Epoch 009 - training loss: 1.3314
2024-09-10 21:48:04 [INFO]: Epoch 010 - training loss: 1.2868
2024-09-10 21:48:04 [INFO]: Finished training. The best model is from epoch#10

In [49]:
brits_impute = saits.impute(dataset)

In [50]:
my_brits_impute = myBritrs.impute(dataset)
indicating_mask = np.isnan(X) ^ np.isnan(X_ori)

In [51]:
indicating_mask = np.isnan(X) ^ np.isnan(X_ori)
indicating_mask=indicating_mask[:100]

In [52]:
from pypots.utils.metrics import calc_mae

calc_mae(brits_impute, np.nan_to_num(X_ori[:100]), indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)


0.6448835102391127

In [53]:
calc_mae(my_brits_impute, np.nan_to_num(X_ori[:100]), indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)

0.6342681050825613

(100, 48, 37)

In [30]:
indicating_mask.shape

(7671, 48, 37)